# Lesson 24. Using methods as if they were functions

In [2]:
def doubleMethod(i: Int) = i * 2
List(1,2,3).map(doubleMethod)

doubleMethod: (i: Int)Int
res1: List[Int] = List(2, 4, 6)


## How to manually convert a method to a function
Attempting to show the value of a method in the REPL will give a handy error message to remember this by:

In [6]:
def triple(i: Int) = i * 3

triple: (i: Int)Int


In [7]:
triple

<console>: 28: error: missing argument list for method triple

As the error message says, follow the method with a _ (also assign it to capture the value in this case):

In [8]:
val tripleFn = triple _

tripleFn: Int => Int = $Lambda$2048/0x0000000840c17840@1f077035


In [9]:
tripleFn(1)

res5: Int = 3


In [10]:
val x = List(1,2,3)
x.map(tripleFn)

x: List[Int] = List(1, 2, 3)
res6: List[Int] = List(3, 6, 9)


In [11]:
x.map(triple)

res7: List[Int] = List(3, 6, 9)


## Example where methods will fail unless converted into a function:

In [15]:
def double(i: Int) = i * 2
def triple(i: Int) = i * 3

double: (i: Int)Int
triple: (i: Int)Int


In [16]:
val functions = Map(
    "2x" -> double,
    "3x" -> triple)

<console>: 28: error: missing argument list for method double

Fixed by converting to functions:

In [17]:
val functions = Map(
    "2x" -> double _,
    "3x" -> triple _)

functions: scala.collection.immutable.Map[String,Int => Int] = Map(2x -> $Lambda$2079/0x0000000840a2b840@5b32f91, 3x -> $Lambda$2080/0x000000084052a040@5137bbd)


Retrieving and 

In [18]:
functions("2x")(3)

res8: Int = 6


# Lesson 25. How to write functions that take functions as input parameters
#### Terminology note -- FIP - function input parameter
## Introduction
Like in C# you are defining the signature of the function you want to accept when dealing with HOFs:

In [2]:
def sayHello(callback: () => Unit) {
    callback()
}

sayHello: (callback: () => Unit)Unit


Since this is a no parameter function with no result its basically looking for a func to run for side effects. As evidenced by the example for using it:

In [4]:
def helloTrite(): Unit = { println("Hello, Trite") }

sayHello(helloTrite)

Hello, Trite


helloTrite: ()Unit


## More parameters, everywhere
Decent example for taking a FIP with some non-function values:

In [6]:
def executeAndPrint(f: (Int, Int) => Int, x: Int, y: Int): Unit =
    println(f(x, y))

def sum(x: Int, y: Int) = x + y
def multiply(x: Int, y: Int) = x * y

executeAndPrint(sum, 3, 11)
executeAndPrint(multiply, 3, 9)

14
27


executeAndPrint: (f: (Int, Int) => Int, x: Int, y: Int)Unit
sum: (x: Int, y: Int)Int
multiply: (x: Int, y: Int)Int


## Taking multiple functions as input parameters
Define a function that takes:
 * (Int, Int) => Int ((x2))
 * Int ((x2))
 * The Ints will be passed to the two FIPs
 * Return the results from the functions as a tuple

#### Attempt:

In [9]:
def myFunc(f1: (Int, Int) => Int, f2: (Int, Int) => Int, x: Int, y: Int): (Int, Int) =
    (f1(x, y), f2(x, y))

def sum(x: Int, y: Int) = x + y
def multiply(x: Int, y: Int) = x * y

myFunc(sum, multiply, 9, 27)

myFunc: (f1: (Int, Int) => Int, f2: (Int, Int) => Int, x: Int, y: Int)(Int, Int)
sum: (x: Int, y: Int)Int
multiply: (x: Int, y: Int)Int
res4: (Int, Int) = (36,243)


#### Following along with the book:

In [11]:
def execTwoFunctions(f1:(Int, Int) => Int,
                     f2:(Int, Int) => Int,
                     a: Int,
                     b: Int): Tuple2[Int, Int] =
    (f1(a,b), f2(a,b))

def sum(x: Int, y: Int) = x + y
def multiply(x: Int, y: Int) = x * y

execTwoFunctions(sum, multiply, 2, 10)

execTwoFunctions: (f1: (Int, Int) => Int, f2: (Int, Int) => Int, a: Int, b: Int)(Int, Int)
sum: (x: Int, y: Int)Int
multiply: (x: Int, y: Int)Int
res5: (Int, Int) = (12,20)


# 26. How to write a 'map' function
The author outlines a standardized way to approach writing HOFs
#### 1. Accurately state the problem as a sentence
"I want to write a map function that can be used to apply other functions to each element in a List[Int] that it's given."
#### 2. Sketch the function signature:

In [25]:
def map[A,B](f: A => B, list: Seq[A]): Seq[B] = ???
42 // returning a value to prevent the cell output vomiting everywhere

map: [A, B](f: A => B, list: Seq[A])Seq[B]
res13: Int = 42


#### 3. Figure out the function body:

In [20]:
def map[A,B](f: A => B, list: Seq[A]): Seq[B] = {
    for {
        x <- list
    } yield f(x)
}

map: [A, B](f: A => B, list: Seq[A])Seq[B]


## Exercise: write a filter function


In [42]:
def filter[A](f: A => Boolean, list: Seq[A]): Seq[A] = {
    for {
        x <- list if f(x)
    } yield x
}

def isEven(x: Int): Boolean =
    x % 2 == 0

filter(isEven, Range(1,21))

filter: [A](f: A => Boolean, list: Seq[A])Seq[A]
isEven: (x: Int)Boolean
res26: Seq[Int] = Vector(2, 4, 6, 8, 10, 12, 14, 16, 18, 20)


In [40]:
Range(1,20)

res24: scala.collection.immutable.Range = Range 1 until 20


# 27. How to use by-name parameters
Sometimes the goal of accepting a function is to pass a larger block of code, which feels like it would be wrong to call that a HOF for some reason. Since there's apparently a special name for that it hopefully means that feeling was correct and a good sign.
#### This technique can be referred to as either a "by-name" or a "call by-name" parameter apparently